In [1]:
import numpy as np
import torch 
import pandas as pd
import torchvision
from torch import optim,nn

In [2]:
train_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\train.csv',encoding='big5')
test_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\test.csv',encoding='big5',header=None)

In [3]:
train_data.shape

(4320, 27)

In [4]:
test_data.shape

(4320, 11)

In [5]:
train_data.head()

,日期,測站,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5


In [6]:
test_data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,id_0,AMB_TEMP,21,21,20,20,19,19,19,18,17
1,id_0,CH4,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.8
2,id_0,CO,0.39,0.36,0.36,0.4,0.53,0.55,0.34,0.31,0.23
3,id_0,NMHC,0.16,0.24,0.22,0.27,0.27,0.26,0.27,0.29,0.1
4,id_0,NO,1.3,1.3,1.3,1.3,1.4,1.6,1.2,1.1,0.9


In [7]:
train_data1=train_data.iloc[:,3:]

 - split train_data1 to 18 ,每个月20天,每天24小时,每小时18个特征
 - transform to 18,24*20

In [8]:
train_data1=train_data1.replace('NR',0)

In [9]:
month_data={}
for m in range(12):
    sample_data=np.empty((18,480),dtype=np.float)
    for d in range(20):
        sample_data[:,24*d:24*(d+1)]=train_data1.iloc[18*(m*20+d):18*(m*20+d+1),:]
    month_data[m]=sample_data

In [10]:
month_data[0].shape

(18, 480)

- from 9 hour data to predict No. 10 hour data
- split group per 9 hours data 
- 20 * 24 -9 = 471 groups per month
- 471,9 * 18 per month
- 12 * 471,9*18

In [11]:
x=np.empty((12*471,9*18))
y=np.empty((12*471,1))

In [12]:
for m in range(12):
    for d in range(471):
#         print(m,d)
#         print(month_data[m][:,9*d:9*(d+1)].shape)
        x[m*471+d:m*471+(d+1),:]=month_data[m][:,d:9+d].reshape(1,-1)
        y[m*471+d:m*471+(d+1),0]=month_data[m][9,9+d]

In [13]:
x1=pd.DataFrame(x)
x1=x1.apply(lambda x: (x-x.mean())/x.std())
x1.head()

,0,1,2,3,4,5,6,7,8,9,...,152,153,154,155,156,157,158,159,160,161
0,-1.358133,-1.358719,-1.359102,-1.517992,-1.676543,-1.675830,-1.675028,-1.674061,-1.197040,0.775375,...,-1.598482,-1.138872,-0.764102,-1.044615,-1.324452,-1.043850,0.173173,0.266484,0.265656,-1.140720
1,-1.358133,-1.358719,-1.518065,-1.676883,-1.676543,-1.675830,-1.675028,-1.198262,-0.880015,0.775375,...,-1.411100,-0.763602,-1.045549,-1.325864,-1.043502,0.173616,0.266881,0.266484,-1.139531,-1.328212
2,-1.358133,-1.517759,-1.677028,-1.676883,-1.676543,-1.675830,-1.198999,-0.881062,-0.404478,0.775375,...,-0.661572,-1.045054,-1.326996,-1.044615,0.173945,0.267267,0.266881,-1.139134,-1.326889,-0.859484
3,-1.517230,-1.676800,-1.677028,-1.676883,-1.676543,-1.199572,-0.881646,-0.405263,-0.087454,0.775375,...,-0.380499,-1.326506,-1.045549,0.174130,0.267594,0.267267,-1.138731,-1.326549,-0.858493,-0.484501
4,-1.676327,-1.676800,-1.677028,-1.676883,-1.200071,-0.882066,-0.405616,-0.088063,-0.087454,0.775375,...,-0.193117,-1.045054,0.174053,0.267880,0.267594,-1.137501,-1.326146,-0.858010,-0.483777,0.265463


In [15]:
x2=torch.Tensor(x1.values)
y2=torch.Tensor(y)
w=torch.Tensor(np.random.normal(0,0.01,(x2.shape[1],1)))
b=torch.Tensor([1.])
for p in [w,b]:
    p.requires_grad_(requires_grad=True)


model by hand

In [16]:
def lin_reg(x,w,b):
    return torch.mm(x,w)+b
    

In [17]:
def squared_loss(y_hat,y):
    return (y_hat-y.view(y_hat.size()))**2

In [18]:
def sgd(params,lr,batch_size):
    for p in params:
        p.data -= lr * p.grad/batch_size

In [19]:
def train_hand(num_epochs,lr,train_features,train_labels,batch_size):
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    for i in range(num_epochs):
        for x,y in data_iter:
            l=squared_loss(lin_reg(x,w,b),y).mean()
            if w.grad is not None:
                w.grad.data.zero_()
                b.grad.data.zero_()
            l.backward()
            sgd([w,b],lr,batch_size)
        if (i+1) % 100 ==0:
            with torch.no_grad():
                print('epoch ',i+1,'train loss: ',squared_loss(lin_reg(train_features,w,b),train_labels).mean().item())
            

In [20]:
x2=torch.Tensor(x1.values)
y2=torch.Tensor(y)
w=torch.Tensor(np.random.normal(0,0.01,(x2.shape[1],1)))
b=torch.Tensor([1.])
for p in [w,b]:
    p.requires_grad_(requires_grad=True)

train_hand(10000,0.01,x2,y2,256)

epoch  100 train loss:  380.9410705566406
epoch  200 train loss:  279.9990539550781
epoch  300 train loss:  211.99411010742188
epoch  400 train loss:  164.7173614501953
epoch  500 train loss:  131.41778564453125
epoch  600 train loss:  107.8755874633789
epoch  700 train loss:  91.11438751220703
epoch  800 train loss:  79.11027526855469
epoch  900 train loss:  70.45252227783203
epoch  1000 train loss:  64.12333679199219
epoch  1100 train loss:  59.45973205566406
epoch  1200 train loss:  56.00725555419922
epoch  1300 train loss:  53.378814697265625
epoch  1400 train loss:  51.36820983886719
epoch  1500 train loss:  49.788265228271484
epoch  1600 train loss:  48.51777648925781
epoch  1700 train loss:  47.483455657958984
epoch  1800 train loss:  46.62392044067383
epoch  1900 train loss:  45.89391326904297
epoch  2000 train loss:  45.26203918457031
epoch  2100 train loss:  44.70806884765625
epoch  2200 train loss:  44.20965576171875
epoch  2300 train loss:  43.76535415649414
epoch  2400 tra

model by nn

In [21]:
def lr_net(feat_input,feat_output):
    return nn.Linear(feat_input,feat_output)

In [22]:
def train_pytorch(num_epochs,lr,train_features,train_labels,batch_size):
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    net=nn.Linear(train_features.shape[1],1)
    loss=nn.MSELoss()
    optimizer=optim.SGD(net.parameters(),lr=lr)
    for e in range(num_epochs):
        for x,y in data_iter:
            

SyntaxError: unexpected EOF while parsing (<ipython-input-22-f459b0cd9ac3>, line 9)